In [ ]:
'''from google.colab import drive
drive.mount('/content/drive')'''

In [ ]:
#!pip install catboost

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from scipy.stats import pearsonr

from sklearn.preprocessing import MinMaxScaler, StandardScaler

In [ ]:
myfolder = "../data/CMaps/"

# **Columns' names**

In [ ]:
#Columns' names
'''
1)  unit number
2)	time, in cycles
3)	operational setting 1
4)	operational setting 2
5)	operational setting 3
6)	sensor measurement  1
7)	sensor measurement  2
...
26)	sensor measurement  21
'''
unitNames = ['UnitNumber']
timeCycles = ["TimeInCycles"]
operSets = ["OperSet"+str(i) for i in range(1,4)] # 1,2 et 3
sensorMes = ["SensorMes"+str(j) for j in range(1, 22)] # de 1 à 21
columnsNames = unitNames + timeCycles + operSets +sensorMes

# **Datasets loading**

In [ ]:
def data_loading(x):
  train_path = myfolder + "train_"+ x +".txt"
  test_path = myfolder + "test_"+ x +".txt"
  rul_path = myfolder + "RUL_"+ x +".txt"
  train = pd.read_csv(train_path, delim_whitespace=True, names=columnsNames)
  test = pd.read_csv(test_path, delim_whitespace=True, names=columnsNames)
  rul = pd.read_csv(rul_path, delim_whitespace=True, names=["RUL_FD"])
  return train, test, rul

train_fd001, test_fd001, rul_fd001 = data_loading("FD001")
train_fd002, test_fd002, rul_fd002 = data_loading("FD002")
train_fd003, test_fd003, rul_fd003 = data_loading("FD003")
train_fd004, test_fd004, rul_fd004 = data_loading("FD004")

In [ ]:
# Forcer l'affichage de toutes les colonnes
pd.set_option('display.max_columns', None)

train_fd004.head(3)

# **RUL column generation for train and test set**

In [ ]:
def rul_train_generation(x):
  rul = pd.DataFrame(x.groupby('UnitNumber')['TimeInCycles'].max()).reset_index()
  rul.columns = ['UnitNumber', 'max']
  x = x.merge(rul, on=['UnitNumber'], how='left')
  x['RUL'] = x['max'] - x['TimeInCycles']
  x.drop('max', axis=1, inplace=True)
  return x

train_fd001 = rul_train_generation(train_fd001)
train_fd002 = rul_train_generation(train_fd002)
train_fd003 = rul_train_generation(train_fd003)
train_fd004 = rul_train_generation(train_fd004)

In [ ]:
train_fd004.head(3)

In [ ]:
def rul_test_generation(x, rul):

  rul["UnitNumber"] = rul.index + 1 # +1 pour que UnitNumber demarre de 1 au lieu de 0, car il s'agit du numero des moteur

  x = x.merge(rul, on=['UnitNumber'], how='left')

  max_cycle = pd.DataFrame(x.groupby('UnitNumber')['TimeInCycles'].max()).reset_index()
  max_cycle.columns = ['UnitNumber', 'max']
  x = x.merge(max_cycle, on=['UnitNumber'], how='left')
  x['RUL'] = x['RUL_FD'] + x['max'] - x['TimeInCycles']
  x.drop(['max', 'RUL_FD'], axis=1, inplace=True)

  return x

test_fd001 = rul_test_generation(test_fd001, rul_fd001)
test_fd002 = rul_test_generation(test_fd002, rul_fd002)
test_fd003 = rul_test_generation(test_fd003, rul_fd003)
test_fd004 = rul_test_generation(test_fd004, rul_fd004)

In [ ]:
test_fd004.head(3)

In [ ]:
print("train_FD001: ",train_fd001.shape, ". test_FD001: ", test_fd001.shape,". RUL: ", rul_fd001.shape)
print("train_FD002: ",train_fd002.shape, ". test_FD002: ", test_fd002.shape,". RUL: ", rul_fd002.shape)
print("train_FD003: ",train_fd003.shape, ". test_FD003: ", test_fd003.shape,". RUL: ", rul_fd003.shape)
print("train_FD004: ",train_fd004.shape, ". test_FD004: ", test_fd004.shape,". RUL: ", rul_fd004.shape)

In [ ]:
print("train_FD001: ",train_fd001.shape, ". test_FD001: ", test_fd001.shape,". nbr moteurs", len(train_fd001['UnitNumber'].unique()))
print("train_FD002: ",train_fd002.shape, ". test_FD002: ", test_fd002.shape,". nbr moteurs", len(train_fd002['UnitNumber'].unique()))
print("train_FD003: ",train_fd003.shape, ". test_FD003: ", test_fd003.shape,". nbr moteurs", len(train_fd003['UnitNumber'].unique()))
print("train_FD004: ",train_fd004.shape, ". test_FD004: ", test_fd004.shape,". nbr moteurs", len(train_fd004['UnitNumber'].unique()))

In [ ]:
len(train_fd002['UnitNumber'].unique())

# **Data normalization**

In [ ]:
def normalised_df(train, test):

  from sklearn.preprocessing import MinMaxScaler

  # Instancier l'objet MinMaxScaler pour normaliser les données
  scaler = MinMaxScaler()

  # Normaliser train
  train_scaled = scaler.fit_transform(train)
  train_df = pd.DataFrame(train_scaled)

  # Normaliser test
  test_scaled = scaler.fit_transform(test)
  test_df = pd.DataFrame(test_scaled)


  train_df.columns = train.columns
  test_df.columns = test.columns
    
  return train_df, test_df

In [ ]:
train_fd001, test_fd001 = normalised_df(train_fd001, test_fd001)
train_fd002, test_fd002 = normalised_df(train_fd002, test_fd002)
train_fd003, test_fd003 = normalised_df(train_fd003, test_fd003)
train_fd004, test_fd004 = normalised_df(train_fd004, test_fd004)

In [ ]:
train_fd004.head(3)

# **Data splitting**

In [ ]:
def data_split(train, test):

  # data split
  X_train = train.drop('RUL', axis=1)
  Y_train = train['RUL']
  X_test = test.drop('RUL', axis=1)
  Y_test = test['RUL']
    
  return X_train, Y_train, X_test, Y_test


# **The KNN Regressor model**

In [ ]:
def my_knn_regressor(train, test):

  X_train, Y_train, X_test, Y_test = data_split(train, test)

  # Créer le modèle KNN Regressor
  model = KNeighborsRegressor()

  # Définir les paramètres à tester dans la recherche par grille
  param_grid = {
      'n_neighbors': np.arange(1, 100),
      'weights': ['uniform', 'distance'],
      'metric': ['euclidean', 'manhattan', 'minkowski', 'chebyshev']
      }

  # Créer l'objet GridSearchCV
  grid_search = GridSearchCV(model, param_grid, cv=5, scoring='neg_mean_squared_error')

  # Effectuer la recherche par grille sur les données d'entraînement
  grid_search.fit(X_train, Y_train)

  # Afficher les meilleurs paramètres trouvés
  print("Meilleurs paramètres trouvés :")
  print(grid_search.best_params_)

  #-------------Train---------------------
  # Prédire les valeurs en utilisant le modèle KNN Regressor pour les données Train
  y_pred_train = grid_search.predict(X_train)
  # Afficher l'erreur quadratique moyenne et le coefficient de détermination R2
  mse_train = mean_squared_error(Y_train, y_pred_train)
  rmse_train = np.sqrt(mse_train)
  mae_train = mean_absolute_error(Y_train, y_pred_train)
  mape_train = np.mean(np.abs((Y_train - y_pred_train) / Y_train)) * 100
  r2_train = r2_score(Y_train, y_pred_train)
  adjusted_r2_train = 1 - (1-r2_train)*(len(Y_train)-1)/(len(Y_train)-X_train.shape[1]-1)

  print('\n=============================Train=============================')
  print('MSE : ',mse_train * 100,'%')
  print('RMSE : ',rmse_train * 100,'%')
  print('MAE : ',mae_train * 100,'%')
  print('MAPE : ',mape_train,'%')
  print('R2 : ',r2_train * 100,'%')
  print('Adjusted R2 : ', adjusted_r2_train * 100,'%')

  #-------------Test---------------------
  # Prédire les valeurs en utilisant le modèle KNN Regressor pour les données Test
  y_pred_test = grid_search.predict(X_test)
  # Afficher l'erreur quadratique moyenne et le coefficient de détermination R2
  mse_test = mean_squared_error(Y_test, y_pred_test)
  rmse_test = np.sqrt(mse_test)
  mae_test = mean_absolute_error(Y_test, y_pred_test)
  mape_test = np.mean(np.abs((Y_test - y_pred_test) / Y_test)) * 100
  r2_test = r2_score(Y_test, y_pred_test)
  adjusted_r2_test = 1 - (1-r2_test)*(len(Y_test)-1)/(len(Y_test)-X_test.shape[1]-1)

  print('\n=============================Test=============================')
  print('MSE : ',mse_test * 100,'%')
  print('RMSE : ',rmse_test * 100,'%')
  print('MAE : ',mae_test * 100,'%')
  print('MAPE : ',mape_test,'%')
  print('R2 : ',r2_test * 100,'%')
  print('Adjusted R2 : ', adjusted_r2_test * 100,'%')

In [ ]:
my_knn_regressor(train_fd001, test_fd001)

In [ ]:
my_knn_regressor(train_fd002, test_fd002)

In [ ]:
my_knn_regressor(train_fd003, test_fd003)

In [ ]:
my_knn_regressor(train_fd004, test_fd004)